In [15]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import os
import importlib
import numpy as np
import matplotlib.pyplot as plt
import shutil
import torchvision.transforms as transforms
import json
import cv2
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
cuda = torch.device('cuda')


## Parameters

In [16]:
# Parameters to find files
PATH_TO_FOLDER_WITH_DATASETS = "/home/lorenzo/datasets/new_dataset/training_data/"
MODEL_SAVE_FOLDER = "/home/lorenzo/models/fast_tunnel_traversal"
MODEL_NAME = "dir_to_axis_data_aug_larger_dataset_vld"
# Parameters to choose the dataset
DATASET_PARAMETERS = {
    "conversor/img_size":100,
    "conversor/max_coord_val":10,
    "label_distance":5,
    "dataset_type":"dir_to_axis",
    "number_of_samples_per_env":10000,
    }
# Network Parameters
MODULE_TO_IMPORT_NETWORK = "tunnel_yaw_prediction.models"
NETWORK_CLASS_NAME = "TunnelYawPredictor"
N_EPOCHS = 64
BATCH_SIZE = 128
LR = 0.001
DO_DATA_AUG = False
# Derived paramters
PATH_TO_MODEL = os.path.join(MODEL_SAVE_FOLDER, MODEL_NAME+".torch")
PATH_TO_MODEL_INFO = os.path.join(MODEL_SAVE_FOLDER, MODEL_NAME+".json")
MODULE = importlib.import_module(MODULE_TO_IMPORT_NETWORK)
MODEL = getattr(MODULE,NETWORK_CLASS_NAME)
os.makedirs(MODEL_SAVE_FOLDER,exist_ok=True)

### Check datasets available in data folder

In [17]:
matching_datasets = []
for dataset_name in os.listdir(PATH_TO_FOLDER_WITH_DATASETS):
    path_to_dataset = os.path.join(PATH_TO_FOLDER_WITH_DATASETS,dataset_name)
    dataset_info_file_path = os.path.join(path_to_dataset, "info.json")
    with open(dataset_info_file_path, "r") as f:
        dataset_info = json.load(f)
    matching_datasets.append(dataset_name)
    for dataset_param in DATASET_PARAMETERS:
        if not dataset_param in dataset_info:
            matching_datasets.remove(dataset_name)
            break
        elif dataset_info[dataset_param]!= DATASET_PARAMETERS[dataset_param]:
            matching_datasets.remove(dataset_name)
            break
if len(matching_datasets) == 1:
    path_to_dataset = os.path.join(PATH_TO_FOLDER_WITH_DATASETS,matching_datasets[0])
    path_to_dataset_info = os.path.join(path_to_dataset, "info.json")
    with open(path_to_dataset_info, "r") as f:
        dataset_info = json.load(f)
elif len(matching_datasets) == 0:
    raise Exception("No dataset matches the params")
else:
    print(f"Choose the desired dataset by number")
    for i, dataset_name in enumerate(matching_datasets):
        path_to_dataset_info = os.path.join(PATH_TO_FOLDER_WITH_DATASETS, dataset_name, "info.json")
        with open(path_to_dataset_info, "r") as f:
            dataset_info = json.load(f)
        print(f"{i}: {dataset_name} // info: {dataset_info}")
        print(f"\t -> n_samples: {len(os.listdir(os.path.join(PATH_TO_FOLDER_WITH_DATASETS, dataset_name)))-1}")
    n = input("Number of selected dataset: ")
    n = int(n)
    path_to_dataset = os.path.join(PATH_TO_FOLDER_WITH_DATASETS, matching_datasets[n])
    path_to_dataset_info = os.path.join(path_to_dataset, "info.json")
    with open(path_to_dataset_info, "r") as f:
        dataset_info = json.load(f)
print(f"Loaded dataset at {path_to_dataset:}")

Choose the desired dataset by number
0: 2023-07-14_15:32:29 // info: {'name': '2023-07-14_15:32:29', 'dataset_type': 'dir_to_axis', 'data_folder': '/home/lorenzo/datasets/new_dataset', 'number_of_samples_per_env': 10000, 'max_rel_yaw_deg': 45, 'label_distance': 5, 'max_horizontal_displacement': 1.8, 'min_vertical_displacement': -0.1, 'max_vertical_displacement': 0.1, 'max_inclination_deg': 10, 'robot_name': '/', 'image_topic': '/cenital_image', 'conversor/max_coord_val': 10, 'conversor/img_size': 100}
	 -> n_samples: 119038
1: 2023-07-17_13:30:17 // info: {'name': '2023-07-17_13:30:17', 'dataset_type': 'dir_to_axis', 'data_folder': '/home/lorenzo/datasets/new_dataset', 'number_of_samples_per_env': 10000, 'max_rel_yaw_deg': 45, 'label_distance': 5, 'max_horizontal_displacement': 1.8, 'min_vertical_displacement': -0.1, 'max_vertical_displacement': 0.1, 'max_inclination_deg': 10, 'robot_name': '/', 'image_topic': '/cenital_image', 'conversor/max_coord_val': 10, 'conversor/img_size': 100}


Loaded dataset at /home/lorenzo/datasets/new_dataset/training_data/2023-07-17_13:30:17


## Load dataset

In [18]:
# Define the dataset
class ImageDataset(Dataset):
    def __init__(self, path_to_dataset, dataset_info):
        self.device = torch.device(
            "cuda:0")
        self.load_dataset(path_to_dataset, dataset_info)
        self.data_augmentation=torch.nn.Sequential(
            transforms.RandomErasing()
        )
    def load_dataset(self, path_to_dataset, dataset_info):
        elements_in_dataset = os.listdir(path_to_dataset)
        elements_in_dataset.remove("info.json")
        elements_in_dataset.sort()
        self.n_datapoints = len(elements_in_dataset)
        self.labels = torch.zeros((self.n_datapoints, 1)).float()
        self.images = torch.zeros((self.n_datapoints,1,100,100)).float()
        folder_loop = tqdm(elements_in_dataset,)
        for dtp_n, dtp_name in enumerate(folder_loop):
            dtp_path = os.path.join(path_to_dataset, dtp_name)
            dtp = np.load(dtp_path)
            self.labels[dtp_n, :] = torch.tensor(dtp["label"])
            self.images[dtp_n,0, :,:] = torch.tensor(dtp["image"])

    def __len__(self):
        return self.n_datapoints

    def __getitem__(self, idx):
        image = self.images[idx, ...]
        result = self.labels[idx, ...]
        return image.float(), result.float()
    def delete(self, idx):
        self.images = torch.cat(self.images[0:idx,...],self.images[idx+1,:])
        self.labels= torch.cat(self.labels[0:idx,...],self.labels[idx+1,:])

dataset = ImageDataset(path_to_dataset, dataset_info)

100%|██████████| 360000/360000 [05:12<00:00, 1152.54it/s]


In [19]:
train_dataset, test_dataset = random_split(dataset,[0.9,0.1])
torch.cuda.empty_cache()
train_dataloader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True, num_workers=5)
test_dataloader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True, num_workers=5)
    

# Data augmentation

In [20]:
def add_random_circle_to_img(image:np.ndarray):
    height, width = image.shape
    radius = np.random.randint(1, 10)
    thickness = np.random.randint(1,3)
    center_x = np.random.randint(0,width)
    center_y = np.random.randint(0,height)
    return cv2.circle(image, (center_x, center_y),radius, 1, thickness )

def add_random_line_to_img(image:np.ndarray):
    height, width = image.shape
    thickness = np.random.randint(1,3)
    p1_x = np.random.randint(0,width)
    p1_y = np.random.randint(0,height)
    p2_x = np.random.randint(0,width)
    p2_y = np.random.randint(0,height)
    return cv2.line(image, (p1_x, p1_y), (p2_x, p2_y), 1, thickness)

def add_dot_to_img(image:np.ndarray):
    height, width = image.shape
    center_x = np.random.randint(0,width)
    center_y = np.random.randint(0,height)
    return cv2.circle(image, (center_x, center_y),1, 1, 1)

def data_augment_numpy_image(image:np.ndarray):
    for _ in range(2):
        if np.random.random() >0.5:
            image[0,...] = add_random_circle_to_img(image[0,...])
        if np.random.random() >0.5:
            image[0,...] = add_random_line_to_img(image[0,...])
    for _ in range(30):
        if np.random.random() > 0.5:
            image[0,...] = add_dot_to_img(image[0,...])
    return image
def data_augment_torch_batch(torch_batch:torch.Tensor):
    numpy_batch = torch_batch.detach().numpy()   
    for i in range(len(numpy_batch)):
        numpy_batch[i, ...] = data_augment_numpy_image(numpy_batch[i,...])
    augmented_torch_batch = torch.tensor(numpy_batch)
    return augmented_torch_batch

In [26]:
def basic_train(network:nn.Module, train_loader, test_loader, criterion, optimizer, n_epochs,lr,name,tensorborad_folder="/home/lorenzo/tensor_board",):
    writer = SummaryWriter(log_dir=tensorborad_folder)
    n_iter = 0
    for n_epoch,epoch in enumerate(tqdm(range(n_epochs))):  # loop over the dataset multiple times
        network.train()
        for i, data in enumerate(train_loader):
            torch.cuda.empty_cache()
            inputs, labels = data
            if DO_DATA_AUG: 
                inputs = data_augment_torch_batch(inputs)
            inputs = inputs.to(torch.device("cuda"))
            labels = labels.to(torch.device("cuda"))
            outputs = network(inputs)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            writer.add_scalar(f"every_train_loss",loss,n_iter)
            n_iter+=1
        writer.add_scalar(f"train_loss_per_epoch",loss,n_epoch)
        network.eval() 
        test_losses = np.zeros((0))
        for i, data in enumerate(test_loader):
            inputs, labels = data
            inputs = inputs.to(torch.device("cuda"))
            labels = labels.to(torch.device("cuda"))
            outputs = network(inputs)
            loss = criterion(outputs, labels)
            loss = loss.cpu().detach().numpy()
            test_losses = np.concatenate([test_losses, np.reshape(loss, -1)])
        test_loss = np.mean(test_losses)
        writer.add_scalar(f"test_loss_per_epoch",test_loss,n_epoch)

### Train

In [22]:
if False:
    with open("/home/lorenzo/models/gallery_detection/procedural_datasets/dataset_03/gallery_detector_v3-_r10_lr002_3.torch", "rb") as f:
        network.load_state_dict(torch.load(f))

In [23]:
lrs = [0.1*0.65**n for n in range(20)]
for i in lrs:
    print(f"{i:05f}")

0.100000
0.065000
0.042250
0.027463
0.017851
0.011603
0.007542
0.004902
0.003186
0.002071
0.001346
0.000875
0.000569
0.000370
0.000240
0.000156
0.000102
0.000066
0.000043
0.000028


In [27]:
criterion = nn.MSELoss()
network = MODEL().to(cuda).float()
torch.cuda.empty_cache()
optimizer = torch.optim.Adam(
    network.parameters(),
    lr=LR,
)
loss_hist = basic_train(
    network, train_dataloader,test_dataloader, criterion, optimizer, N_EPOCHS, LR,MODEL_NAME
)
network.eval()
network.to("cpu")
# Calculate the error on the validation data
mean_squared_errors = np.zeros(0)
for test_data in test_dataloader:
    inputs, labels = test_data
    outputs = network(inputs)
    mean_squared_errors = np.hstack([mean_squared_errors,criterion(outputs, labels).detach().numpy()])
mean_squared_error = np.mean(mean_squared_errors)
mean_error_rad = np.sqrt(mean_squared_error)
mean_error_deg = np.rad2deg(mean_error_rad)

100%|██████████| 64/64 [1:00:39<00:00, 56.86s/it]


In [28]:
print(mean_error_deg)

3.4249089162462485


In [30]:
PATH_TO_MODEL = "/home/lorenzo/models/fast_tunnel_traversal/dir_to_axis_data_aug_larger_dataset_vld_2.torch"
PATH_TO_MODEL_INFO = "/home/lorenzo/models/fast_tunnel_traversal/dir_to_axis_data_aug_larger_dataset_vld_2.json"

In [31]:
info_about_training = {
    "path_to_dataset": path_to_dataset,
    "path_to_dataset_info":path_to_dataset_info,
    "model_name":MODEL_NAME,
    "dataset_paramters":DATASET_PARAMETERS,
    "module_to_import_network":MODULE_TO_IMPORT_NETWORK,
    "network_class_name": NETWORK_CLASS_NAME,
    "n_epochs":N_EPOCHS,
    "batch_size":BATCH_SIZE,
    "lr":LR,
    "mean_error":mean_error_rad,
    }
torch.save(network.state_dict(), PATH_TO_MODEL)
with open(PATH_TO_MODEL_INFO,"w") as f:
    json.dump(info_about_training,f)
print(PATH_TO_MODEL)

/home/lorenzo/models/fast_tunnel_traversal/dir_to_axis_data_aug_larger_dataset_vld_2.torch
